In [ ]:
#

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv('../input/vehicle-dataset-from-cardekho/car data.csv')
df.head(9)

In [ ]:
df.shape

In [ ]:
print(df['Seller_Type'].unique())
print(df['Transmission'].unique())
print(df['Owner'].unique())

In [ ]:
df.isnull().sum()#to show null values in each features

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
final_dataset =df [['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]


In [ ]:
final_dataset['Current_Year']=2021

In [ ]:
final_dataset['Number_of_years'] = final_dataset['Current_Year']-final_dataset['Year']

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.drop(['Year'],axis=1,inplace=True)


In [ ]:
final_dataset.head()

In [ ]:
final_dataset.drop(['Current_Year'],axis=1,inplace=True)

In [ ]:
final_dataset.head()

In [ ]:
final_dataset = pd.get_dummies(final_dataset, drop_first = True)

In [ ]:
final_dataset.head(9)

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(final_dataset)

In [ ]:
import matplotlib.pyplot as plt


In [ ]:
corrmat=final_dataset.corr()
top_corr_features = corrmat.index
#plt.figure(figsize=(20,20))

g=sns.heatmap(final_dataset[top_corr_features].corr(),annot=True,cmap="RdYlGn")






In [ ]:
X = final_dataset.iloc[:,1:]
Y = final_dataset.iloc[:,0]

In [ ]:
X.head()
Y.head()

In [ ]:
#graph fore feature importance for better visualizatiom
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(X,Y)


In [ ]:
print(model.feature_importances_)

In [ ]:
feat_importances = pd.Series(model.feature_importances_, index = X.columns)
print(feat_importances)

In [ ]:
feat_importances.nlargest(5).plot(kind = 'barh')
plt.show()

In [ ]:

from sklearn.model_selection import train_test_split
x_train , x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.03, random_state = 0)


In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

In [ ]:
n_estimator = [int(i) for i in np.linspace(start = 100, stop = 1200, num = 12)]
print(n_estimator)

In [ ]:

from sklearn.model_selection import RandomizedSearchCV

In [ ]:
max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
    
min_samples_split = [2, 5, 10, 15, 100]

min_samples_leaf = [1, 2, 5, 10]

In [ ]:

random_grid = {'n_estimators': n_estimator,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}
print(random_grid)

In [ ]:
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_random.fit(x_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
predictions = rf_random.predict(x_test)

In [ ]:
sns.distplot(y_test-predictions)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
import pickle
file  = open('random_forest_regression_model.pkl' , 'wb')
pickle.dump(rf_random,file)